<hr>

# Wave Generation

In [7]:
from data_generators.utils import random_waves as rw
import numpy as np
import pandas as pd
import plotly.express as px

# rw.seed()

# y = rw.half_waves_fixed_freq_varying_amp(num_points=10000, periods=7, amplitude=(3.5, 8.5), phase=5.7, smooth=10, noise=0.1)
# y = rw.full_waves_fixed_freq_varying_amp(num_points=10000, periods=7, amplitude=(3.5, 8.5), phase=5.7, noise=0.1)

y = rw.sinusoidal_wave(num_points=10000, periods=7, amplitude=(5.5, 7.2), wave_mode='full', varying_mode='fixed', phase=5.7, noise=0.1)
# y = rw.sinusoidal_wave(num_points=10000, periods=7, amplitude=(5.5, 7.2), wave_mode='full', varying_mode='fixed')

print(f"num_points: {len(y)}")
# display(y.head())

fig = px.line(y=y, title='Random Wave Example')
fig.update_layout(
    xaxis_title='Index',
    yaxis_title='Amplitude',
    template='plotly_dark'
)
fig.show()

2024-12-03 14:37:35 - main - WARNING - Varying mode is set to only vary frequency but amplitude range is different. Amplitude will be fixed. Using lower bound of amplitude: 5.5
num_points: 10000


In [ ]:
from data_generators.utils import random_waves as rw

print(rw.equals_parts(1000, 3))
print(rw.random_parts(1000, 5))

[333, 333, 334]
[140, 289, 122, 302, 147]
